<a href="https://colab.research.google.com/github/Zanale10/DATASCIENCE-101-TEST/blob/main/mix_link_to_Powerbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import glob
import os

# ✅ Folder path
folder_path = r"C:\Users\USER\OneDrive - DANCO CAPITAL LTD\DANCO OPERATIONS\DANCO PRODUCTION\2025\FLOOR STOCK\ACTUALS 2025"

# 1. Read all Excel files
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
print(f"✅ Found {len(all_files)} Excel files in Actuals folder.")

data = []

for file in all_files:
    print(f"📂 Reading: {file}")

    try:
        # Get available sheets
        xl = pd.ExcelFile(file)
        print("   Sheets available:", xl.sheet_names)

        # Pick sheet with "requirement" in name, else first sheet
        sheet_to_read = next((s for s in xl.sheet_names if "requirement" in s.lower()), xl.sheet_names[0])

        # Read the detected sheet
        df = pd.read_excel(file, sheet_name=sheet_to_read)

        # Set headers correctly
        df.columns = df.iloc[0]      # First row as headers
        df = df[1:]                  # Remove header row

        # Remove cumulative rows
        df = df[~df.iloc[:,0].astype(str).str.contains('Cumulative', case=False, na=False)]

        # Add source file column
        df["Source File"] = os.path.basename(file)

        data.append(df)

    except Exception as e:
        print(f"❌ Error reading {file}: {e}")

# 2. Combine all files
if data:
    master_df = pd.concat(data, ignore_index=True)
    print(f"✅ Combined dataframe shape: {master_df.shape}")

    # 3. Convert numeric columns
    numeric_cols = ['Planned Output', 'Actual Output', 'Material A', 'Material B', 'Scrap']
    master_df[numeric_cols] = master_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # 4. Calculate material mix
    master_df["Total Material"] = master_df[['Material A', 'Material B']].sum(axis=1)
    master_df["Material A Mix %"] = master_df["Material A"] / master_df["Total Material"]
    master_df["Material B Mix %"] = master_df["Material B"] / master_df["Total Material"]

    # 5. Calculate output variance (Actual vs Planned)
    master_df["Output Variance"] = master_df["Actual Output"] - master_df["Planned Output"]

    # 6. Calculate material savings (Planned vs Actual usage)
    master_df["Material Savings"] = (master_df["Planned Output"] - master_df["Actual Output"]) * master_df["Material A Mix %"]

    # 7. Export cleaned file for Power BI
    export_path = r"C:\Users\USER\DANCO CAPITAL LTD\DANCO OPERATIONS\ProductionMaster.csv"
    master_df.to_csv(export_path, index=False)
    print(f"✅ Data exported to: {export_path}")

else:
    print("⚠ No data loaded. Check if files or sheet names are correct.")


✅ Found 0 Excel files in Actuals folder.
⚠ No data loaded. Check if files or sheet names are correct.


In [4]:
import pandas as pd
import glob
import os

folder_path = r"C:\Users\USER\OneDrive - DANCO CAPITAL LTD\DANCO OPERATIONS\DANCO PRODUCTION\2025\FLOOR STOCK\ACTUALS 2025"
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
print(f"Found {len(all_files)} Excel files in Actuals folder.")

data = []

for file in all_files:
    try:
        # Read file, skipping first 2 header rows
        df = pd.read_excel(file, sheet_name="Mtrl Requirement", header=2)

        # Drop specific unwanted rows
        df = df.drop(index=[16], errors='ignore')  # Row 17 (0-based index is 16)
        df = df.drop(index=range(22, 29), errors='ignore')  # Rows 23-29 (0-based index 22-28)

        # Remove rows that are completely blank
        df = df.dropna(how='all')

        # Add filename info for shift/date tracking
        df["Source File"] = os.path.basename(file)

        data.append(df)
    except Exception as e:
        print(f"⚠ Error reading {file}: {e}")

# Combine cleaned data
if data:
    master_df = pd.concat(data, ignore_index=True)
    print(f"✅ Combined {len(master_df)} rows successfully.")

    # Export to CSV for Power BI
    master_df.to_csv(r"C:\Users\USER\OneDrive - DANCO CAPITAL LTD\ProductionMaster.csv", index=False)
    print("📂 Cleaned data exported for Power BI.")
else:
    print("⚠ No valid data found.")


Found 0 Excel files in Actuals folder.
⚠ No valid data found.


In [5]:
import pandas as pd
import glob
import os

folder_path = r"C:\Users\USER\OneDrive - DANCO CAPITAL LTD\DANCO OPERATIONS\DANCO PRODUCTION\2025\FLOOR STOCK\ACTUALS 2025"
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
print(f"Found {len(all_files)} Excel files in Actuals folder.")

data = []

for file in all_files:
    try:
        # Read file, skipping first 2 rows (headers)
        df = pd.read_excel(file, sheet_name="Mtrl Requirement", header=2)

        # Remove unnecessary rows
        df = df.drop(index=[16], errors='ignore')  # Row 17 cumulative
        df = df.drop(index=range(22, 29), errors='ignore')  # Rows 23-29
        df = df.dropna(how='all')

        # Convert numeric columns safely
        numeric_cols = ['Planned Output', 'Actual Output', 'Material A', 'Material B']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

        # Calculate actual material mix
        df["Total Material Used"] = df[['Material A', 'Material B']].sum(axis=1)
        df["Material A Mix %"] = df["Material A"] / df["Total Material Used"]
        df["Material B Mix %"] = df["Material B"] / df["Total Material Used"]

        # Add filename for shift/date tracking
        df["Source File"] = os.path.basename(file)

        data.append(df)
    except Exception as e:
        print(f"⚠ Error reading {file}: {e}")

# Combine all files
if data:
    master_df = pd.concat(data, ignore_index=True)
    print(f"✅ Combined {len(master_df)} rows successfully.")

    # OPTIONAL: Join with planned mix/cost data (if available)
    # Example: master_df = master_df.merge(planned_mix_df, on="Job/Shift ID", how="left")

    # Export cleaned dataset for Power BI
    master_df.to_csv(r"C:\Users\USER\OneDrive - DANCO CAPITAL LTD\ProductionMaster.csv", index=False)
    print("📂 Cleaned data exported for Power BI.")
else:
    print("⚠ No valid data found.")


Found 0 Excel files in Actuals folder.
⚠ No valid data found.
